This Jupyter notebook demonstrate the work done in ["Assessment of individualised dynamic environmental exposures within The World Avatar" by Chen et al., Technical Report 337, c4e-Preprint Series, Cambridge, 2025.](https://como.ceb.cam.ac.uk/preprints/337/).

The complete codes developed for this work are available under an open-source licence on GitHub in The World Avatar repository [https://github.com/cambridge-cares/TheWorldAvatar](https://github.com/cambridge-cares/TheWorldAvatar).

The notebook serves as a data exploration interface and may not generalize to any trajectory IRI provided. For general use case, please refer to the complete code in The World Avatar repository.

# Configuration

## Environment Data
Environment data from TWA knowledge graph is accessed through SPARQL endpoint. The endpoint is pre-configured at `config.env` and can be changed to other endpoints as needed.

## Trajectory Data
User is expected to provide the trajctory data in Postgres database and input the database configuration in `config.env`. 
- DB_HOST
- DB_PORT
- DB_DBNAME
- DB_USER
- DB_PASSWORD

Data is required to store in the same format as described in [here](https://github.com/TheWorldAvatar/baselib/tree/main/src/main/java/uk/ac/cam/cares/jps/base/timeseries#instantiation-in-rdb-using-timeseriesrdbclient). 

User may choose to 
- collect data with [TWA TimelineApp](https://github.com/cambridge-cares/TheWorldAvatar/tree/main/Apps/TimelineApp), which is compatible with the code in this notebook and the full version.
- instantiate data with CSV as describing in [here](https://github.com/cambridge-cares/TheWorldAvatar/tree/main/Agents/FenlandTrajectoryAgent#31-placing-the-gps-data)

## Template
SPARQL and SQL queries are provided as template files in `template/`. No additional configuration is needed.

#  Setup TWA connection

In [ ]:
from twa.kg_operations import PySparqlClient
from twa.conf import config_generic, Config

from typing import Optional, Tuple, List, Dict
import psycopg2
import pandas as pd
import os

import matplotlib.pyplot as plt
from shapely import wkb, wkt
from shapely.geometry import Point, LineString, Polygon
from pyproj import CRS, Transformer, Geod
from shapely.ops import transform
from shapely.wkb import loads as wkb_loads
from shapely.strtree import STRtree
from shapely.affinity import translate
import re
import binascii

In [ ]:
class EnvConfig(Config):
    SPARQL_QUERY_ENDPOINT: str
    TEMPLATE_DIR: str
    DB_HOST: str
    DB_PORT: str
    DB_DBNAME: str
    DB_USER: str
    DB_PASSWORD: str

In [ ]:
CONFIG_DIR = os.getenv("CONFIG_DIR", 'config.env')
env_confg = config_generic(EnvConfig, env_file=CONFIG_DIR)

sparql_client = PySparqlClient(
    query_endpoint = env_confg.SPARQL_QUERY_ENDPOINT,
    update_endpoint = ""
)

template_dir = env_confg.TEMPLATE_DIR

db_params = {
    "host": env_confg.DB_HOST,
    "port": env_confg.DB_PORT,
    "dbname": env_confg.DB_DBNAME,
    "user": env_confg.DB_USER,
    "password": env_confg.DB_PASSWORD
}

exposure_radius = 100

# Pull data from TWA

Exposure calculation util

In [ ]:
def load_template(filename: str) -> str:
    filepath = os.path.join(template_dir, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def execute_query(query: str, params: Optional[tuple] = None):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query, params)
            results = cur.fetchall()
            return results

## Fetch environment data from knowledge graph

Fetch food retails data for the year 2025 in Cambridge, UK.

In [ ]:
food_hygiene_iri = "http://www.theworldavatar.com/ontology/OntoFHRS/FoodHygieneRating"
food_hygiene_rating_template = load_template('get_food_hygiene_rating.sparql')
food_hygiene_rating = pd.DataFrame(sparql_client.perform_query(food_hygiene_rating_template))
food_hygiene_rating.head()

Fetch greenspace data for the year 2023 in Cambridge, UK.

In [ ]:

green_space_template = load_template("get_greenspace_area.sparql")
green_space = pd.DataFrame(sparql_client.perform_query(green_space_template))
green_space.head()

## Fetch trajectory from database

In [ ]:
def get_table_name_for_timeseries(timeseriesIRI: str) -> str:
    try:
        query_template = load_template("get_table_name_for_timeseries.sql")
        rows = execute_query(query_template, (timeseriesIRI,))
        if rows:
            return rows[0][0]
    except Exception as e:
        print(f"[Fallback] Primary get_table_name failed: {e}")

    # Try fallback
    fallback_template = load_template("get_table_name_for_timeseries_fallback.sql")
    rows = execute_query(fallback_template, (timeseriesIRI,))
    if not rows:
        raise ValueError(f"No fallback tableName found for timeseriesIRI: {timeseriesIRI}")
    return rows[0][0]

def get_timeseries_data(table_name: str, trajectory_iri: str) -> pd.DataFrame:
    """
    Retrieve raw timeseries data from database.
    Automatically detects whether 'Trip index' column exists and selects the correct SQL template.
    Returns DataFrame with appropriate columns.
    """
    try:
        check_sql = load_template("check_trip_index_column.sql")
        result = execute_query(check_sql, (table_name,))
        has_trip_index = bool(result)
    except Exception as e:
        print(f"[Trip index] Failed to check for 'Trip index' column: {e}")
        has_trip_index = False

    try:
        if has_trip_index:
            print(f"[Trip index] Detected in table: {table_name}")
            sql_template_name = "get_timeseries_data_with_trip.sql"
        else:
            print(f"[Trip index] Not found, using fallback SQL.")
            sql_template_name = "get_timeseries_data.sql"

        query_template = load_template(sql_template_name)
        query = query_template.format(table_name=table_name)
        rows = execute_query(query)

        base_cols = ["time", "column1", "column2", "column3", "column4", "column5", "column6", "column7"]
        if has_trip_index:
            base_cols.append("Trip index")

        return pd.DataFrame(rows, columns=base_cols)

    except Exception as primary_exception:
        print(f"[Fallback] Primary SQL failed for {table_name}: {primary_exception}")
        
        # Run fallback version
        try:
            fallback_template = load_template("get_timeseries_data_fallback.sql")
            query = fallback_template.format(table_name=table_name)
            rows = execute_query(query, (trajectory_iri,))

            return pd.DataFrame(rows, columns=["time", "LATITUDE", "LONGITUDE"])

        except Exception as fallback_exception:
            print(f"[Fallback] Failed for {table_name}: {fallback_exception}")
            raise fallback_exception


In [ ]:
trajectory_iri = 'https://www.theworldavatar.com/kg/ontotimeseries/Timeseries_f93edf67-8144-4cea-a57d-42cd1cae7fb8'
table_name = get_table_name_for_timeseries(trajectory_iri)
trajectory_df = get_timeseries_data(table_name, trajectory_iri).sort_values(by='time', ascending=True) 

print(f'Trajectory contains {len(trajectory_df)} points')
trajectory_df.head()


# Process trajecotry data


## Group as trips

In [ ]:
# trajectory buffer
if 'Trip index' in trajectory_df.columns:
    trip_index_col = [col for col in trajectory_df.columns if col.lower().replace(" ", "_") == "trip_index"][0]
    trip_df = trajectory_df[trajectory_df[trip_index_col].fillna(0).astype(int) > 0].sort_values(by='time', ascending=True)
    grouped = trip_df.groupby(trip_index_col)

    print(f'Group of trips: {grouped.ngroups}')
else:
    trip_df = trajectory_df.copy().sort_values(by='time', ascending=True)
    trip_df["trip_index"] = 1
    grouped = trip_df.groupby("trip_index")
    print(f'Group of trips: {grouped.ngroups}')

In [ ]:
grouped.head()

In [ ]:
for name, group_df in grouped:
    coords = [
        (pt.x, pt.y)
        for pt in (wkb.loads(bytes.fromhex(hex_str)) for hex_str in list(group_df['column7']))
    ]
    x, y = zip(*coords)
    plt.scatter(x, y, label=f"Trip {name}")
    
plt.title("Trips")
plt.xlabel("Longitude")
plt.ylabel("Latitude") 
plt.grid(True)  
plt.legend()
plt.show()

## Buffer trajctory

In [ ]:
group_processes = {}
coords_whole_trajectory = [
        (pt.x, pt.y)
        for pt in (wkb.loads(bytes.fromhex(hex_str)) for hex_str in list(trajectory_df['column7']))
    ]

for name, group_df in grouped:
    coords = [
        (pt.x, pt.y)
        for pt in (wkb.loads(bytes.fromhex(hex_str)) for hex_str in list(group_df['column7']))
    ]
    base_geom = LineString(coords)
    
    # centroid = base_geom.centroid
    centroid = LineString(coords_whole_trajectory).centroid
    aeqd_crs = CRS.from_proj4(f"+proj=aeqd +lat_0={centroid.y} +lon_0={centroid.x} +units=m")
    to_aeqd = Transformer.from_crs(4326, aeqd_crs, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(aeqd_crs, 4326, always_xy=True).transform

    proj_base = transform(to_aeqd, base_geom)

    group_processes[name] = {
        "aeqd_crs": aeqd_crs,
        "to_aeqd": to_aeqd,
        "to_wgs84": to_wgs84,
        "buffer_proj": proj_base.buffer(exposure_radius)
        }

In [ ]:
print(len(group_processes))

# Exposure Calculation

## Greenspace exposure

Demonstrate area environement features exposure

### 1. Process environment data
Project area features to the same plane as the trajectory

In [ ]:
def fetch_env_crs(env_data_iri: str, feature_type: str) -> str:
    """
    Returns a CRS URI for a geometry; falls back to 4326 for POINT if not found.
    """
    # Minimal change: directly load template
    template = load_template("get_crs_for_env_data.sparql")
    sparql = template.format(env_data_iri=env_data_iri)
    crs = sparql_client.perform_query(sparql)
    
    if crs:
        return crs[0]['crs']

    # If not found or error, use default value
    if feature_type.upper() == "POINT":
        return "http://www.opengis.net/def/crs/EPSG/0/4326"
    return "http://www.opengis.net/def/crs/EPSG/0/4326"

In [ ]:
crs_uri = fetch_env_crs("https://www.theworldavatar.com/kg/ontogreenspace/Greenspace", "AREA")
m = re.search(r"EPSG/0/(\d+)", crs_uri)
src_epsg = int(m.group(1)) if m else 4326

for process_info in group_processes:
    to_env = Transformer.from_crs(f"EPSG:{src_epsg}", group_processes[process_info]['aeqd_crs'], always_xy=True).transform
    proj_greenspace = []
    for _, row in green_space.iterrows():
        wkb_hex = row.get("geometry", "")
        if not wkb_hex or wkb_hex == "N/A":
            continue

        try:
            poly_src = wkb_loads(binascii.unhexlify(wkb_hex), hex=True)
            proj_greenspace.append((transform(to_env, poly_src), None))
        except Exception:
            continue
    group_processes[process_info]['proj_greenspace'] = proj_greenspace

### 2. Trip cumulative exposure with STRtree

In [ ]:
def geodesic_area(geom) -> float:
    """Calculate actual surface area in WGS84 coordinate system (unit: square meters)"""
    if geom is None or geom.is_empty:
        return 0.0
    geod = Geod(ellps='WGS84')
    if geom.geom_type == 'Polygon':
        lons, lats = zip(*list(geom.exterior.coords))
        area, _ = geod.polygon_area_perimeter(lons, lats)
        return abs(area)
    elif geom.geom_type == 'MultiPolygon':
        return sum(geodesic_area(poly) for poly in geom.geoms)
    else:
        return 0.0

In [ ]:
tree = STRtree([g for g, _ in proj_greenspace])
results = []
for trip in group_processes:
    buf_proj = group_processes[trip]['buffer_proj']
    to_wgs84 = group_processes[trip]['to_wgs84']

    spatial_ix = tree.query(buf_proj)

    area_count = 0
    total_area = 0.0

    for i in spatial_ix:
        poly_proj, _ = proj_greenspace[i]
        inter_proj = poly_proj.intersection(buf_proj)
        if inter_proj.is_empty:
            continue
        inter_ll = transform(to_wgs84, inter_proj)
        total_area += geodesic_area(inter_ll)
        area_count += 1
    results.append({'tirp': trip, 'area_count': area_count, 'total_area': total_area})

results

In [ ]:
sum(item['area_count'] for item in results)

In [ ]:
sum(item['total_area'] for item in results)

### 3. Trajectory pointwise exposure

In [ ]:
proj_env = proj_greenspace
tree = STRtree([g for g, _ in proj_greenspace])
def pointwise_area_exposure(row):
    to_aeqd = group_processes[row['Trip index']]['to_aeqd']
    to_wgs84 = group_processes[row['Trip index']]['to_wgs84']
    geom = row['column7']
    
    lon, lat = wkb.loads(bytes.fromhex(geom)).x, wkb.loads(bytes.fromhex(geom)).y
    pt_proj = Point(to_aeqd(lon, lat))
    buf_template = Point(0, 0).buffer(exposure_radius)
    circ_proj = translate(buf_template, pt_proj.x, pt_proj.y)
    idxs_pt = tree.query(circ_proj)
    area_m2 = 0.0
    for i in idxs_pt:
        poly_proj, _ = proj_env[i]
        inter_proj = poly_proj.intersection(circ_proj)
        if inter_proj.is_empty:
            continue
        inter_ll = transform(to_wgs84, inter_proj)
        area_m2 += geodesic_area(inter_ll)
    return area_m2

In [ ]:
pointwise_cal_df = trip_df.copy()
pointwise_cal_df['gs_area'] = pointwise_cal_df.apply(pointwise_area_exposure, axis=1)

In [ ]:
pointwise_cal_df.head()

In [ ]:
grouped_result = pointwise_cal_df.groupby(trip_index_col)
fig, ax = plt.subplots(figsize=(10, 6))

for group_name, df in grouped_result:
    ax.plot(df['time'], df['gs_area'], label=f'Trip {group_name}')

ax.set_title("Greenspace exposure area timeseries (m²)")
ax.set_xlabel("Time")
ax.set_ylabel("Green area (m²)")
ax.legend(title="Trip")

plt.tight_layout()
plt.show()

## Food retails exposure

### 1. Process environment features
Project point features to the same plane as the trajectory

In [ ]:
food_hygiene_rating.head()

In [ ]:
for trip in group_processes:
    to_aeqd = group_processes[trip]['to_aeqd']
    proj_food = []
    for _, row in food_hygiene_rating.iterrows():
        try:
            lon, lat = float(row["longitude"]), float(row["latitude"])
            proj_food.append((Point(to_aeqd(lon, lat)), row.get("id", f"{lon},{lat}")))
        except Exception:
            continue
    group_processes[trip]['proj_food'] = proj_food

### 2. STRtree + cumulative exposure
Calculate food retails exposure of the trajectory

In [ ]:
results = []
tree = STRtree([g for g, _ in proj_food])

for trip in group_processes:
    buf_proj = group_processes[trip]['buffer_proj']
    spatial_ix = tree.query(buf_proj)

    point_count = 0

    visited = set()
    for i in spatial_ix:
        pt_proj, uid = proj_food[i]
        if buf_proj.intersects(pt_proj):
            visited.add(uid)
    results.append({'tirp': trip, 'point_count': len(visited)})

In [ ]:
print(f'Food retails exposure count {results}')

### 3. Trajectory pointwise exposure 
Calculate food retails exposure at individual trajectory points

In [ ]:
proj_env = proj_food
tree = STRtree([g for g, _ in proj_food])
def pointwise_point_exposure(row):
    geom = row['column7']
    to_aeqd = group_processes[row['Trip index']]['to_aeqd']
    
    lon, lat = wkb.loads(bytes.fromhex(geom)).x, wkb.loads(bytes.fromhex(geom)).y
    pt_proj = Point(to_aeqd(lon, lat))
    idxs_pt = tree.query(pt_proj.buffer(exposure_radius))
    cnt = sum(
        1 for i in idxs_pt
        if pt_proj.distance(proj_env[i][0]) <= exposure_radius
    )
    return cnt

In [ ]:
pointwise_cal_df = trip_df.copy().sort_values(by='time', ascending=True)
pointwise_cal_df['food_count'] = pointwise_cal_df.apply(pointwise_point_exposure, axis=1)

pointwise_cal_df.head()

In [ ]:
grouped_result = pointwise_cal_df.groupby(trip_index_col)
fig, ax = plt.subplots(figsize=(10, 6))  # Single shared plot

for group_name, df in grouped_result:
    ax.plot(df['time'], df['food_count'], label=f'Trip {group_name}')

ax.set_title("Food retails exposure count")
ax.set_xlabel("Time")
ax.set_ylabel("Food retails")
ax.legend(title="Trip")

plt.tight_layout()
plt.show()